<a href="https://colab.research.google.com/github/Shreya549/Stonks/blob/main/Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing Pmdarima**

In [1]:
!pip install pmdarima

**Importing the libraries**

In [2]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from fbprophet import Prophet
from matplotlib import pyplot as plt
from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_error, mean_squared_error

myfavouritenumber = 13
seed = myfavouritenumber
np.random.seed(seed)

**Reading the dataset and setting the index**

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/Shreya549/Stonks/main/NIFTY50_all.csv")
df.set_index("Date", drop=False, inplace=True)
df.head()

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,,
2007-11-27,2007-11-27,MUNDRAPORT,EQ,440.00,770.00,1050.00,770.0,959.0,962.90,984.72,27294366,2.687719e+15,NaN,9859619.0,0.3612
2007-11-28,2007-11-28,MUNDRAPORT,EQ,962.90,984.00,990.00,874.0,885.0,893.90,941.38,4581338,4.312765e+14,NaN,1453278.0,0.3172
2007-11-29,2007-11-29,MUNDRAPORT,EQ,893.90,909.00,914.75,841.0,887.0,884.20,888.09,5124121,4.550658e+14,NaN,1069678.0,0.2088
2007-11-30,2007-11-30,MUNDRAPORT,EQ,884.20,890.00,958.00,890.0,929.0,921.55,929.17,4609762,4.283257e+14,NaN,1260913.0,0.2735
2007-12-03,2007-12-03,MUNDRAPORT,EQ,921.55,939.75,995.00,922.0,980.0,969.30,965.65,2977470,2.875200e+14,NaN,816123.0,0.2741


In [4]:
c = df.Symbol.value_counts()
rows = c[c>2500].index
df.shape

(230104, 15)

In [5]:
df.Symbol = df.Symbol.apply(lambda x : x if x in rows else np.nan)

In [6]:
df = df.dropna()
df

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,,
2011-06-01,2011-06-01,ASIANPAINT,EQ,3133.40,3145.10,3194.00,3061.50,3170.00,3169.35,3156.21,192349,6.070944e+13,11665.0,89136.0,0.4634
2011-06-02,2011-06-02,ASIANPAINT,EQ,3169.35,3159.00,3171.80,3087.00,3110.05,3105.05,3133.25,211697,6.633002e+13,6690.0,169251.0,0.7995
2011-06-03,2011-06-03,ASIANPAINT,EQ,3105.05,3105.05,3190.00,3099.95,3181.00,3167.80,3153.77,83355,2.628821e+13,5398.0,45782.0,0.5492
2011-06-06,2011-06-06,ASIANPAINT,EQ,3167.80,3160.20,3188.85,3141.25,3169.50,3170.05,3163.77,46291,1.464539e+13,4838.0,20367.0,0.4400
2011-06-07,2011-06-07,ASIANPAINT,EQ,3170.05,3170.10,3229.80,3155.00,3196.00,3206.15,3202.18,82014,2.626232e+13,8868.0,27403.0,0.3341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-24,2020-09-24,ZEEL,EQ,191.80,185.00,195.80,182.85,193.50,190.35,190.14,76316695,1.451102e+15,279397.0,24361908.0,0.3192
2020-09-25,2020-09-25,ZEEL,EQ,190.35,197.55,201.50,183.20,197.30,198.20,192.54,61557172,1.185250e+15,301334.0,6573633.0,0.1068
2020-09-28,2020-09-28,ZEEL,EQ,198.20,198.75,209.75,197.20,207.60,208.75,205.11,34075363,6.989202e+14,171300.0,4578621.0,0.1344


In [7]:
df.shape

(97800, 15)

**Feature Engineering**

In [8]:

df.reset_index(drop=True, inplace=True)
lag_features = ["High", "Low", "Volume", "Turnover", "Trades"]
window1 = 3
window2 = 7
window3 = 30

df_rolled_3d = df[lag_features].rolling(window=window1, min_periods=0)
df_rolled_7d = df[lag_features].rolling(window=window2, min_periods=0)
df_rolled_30d = df[lag_features].rolling(window=window3, min_periods=0)

df_mean_3d = df_rolled_3d.mean().shift(1).reset_index().astype(np.float32)
df_mean_7d = df_rolled_7d.mean().shift(1).reset_index().astype(np.float32)
df_mean_30d = df_rolled_30d.mean().shift(1).reset_index().astype(np.float32)

df_std_3d = df_rolled_3d.std().shift(1).reset_index().astype(np.float32)
df_std_7d = df_rolled_7d.std().shift(1).reset_index().astype(np.float32)
df_std_30d = df_rolled_30d.std().shift(1).reset_index().astype(np.float32)

for feature in lag_features:
    df[f"{feature}_mean_lag{window1}"] = df_mean_3d[feature]
    df[f"{feature}_mean_lag{window2}"] = df_mean_7d[feature]
    df[f"{feature}_mean_lag{window3}"] = df_mean_30d[feature]
    
    df[f"{feature}_std_lag{window1}"] = df_std_3d[feature]
    df[f"{feature}_std_lag{window2}"] = df_std_7d[feature]
    df[f"{feature}_std_lag{window3}"] = df_std_30d[feature]

df.fillna(df.mean(), inplace=True)

df.set_index("Date", drop=False, inplace=True)
df.head()

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble,High_mean_lag3,High_mean_lag7,High_mean_lag30,High_std_lag3,High_std_lag7,High_std_lag30,Low_mean_lag3,Low_mean_lag7,Low_mean_lag30,Low_std_lag3,Low_std_lag7,Low_std_lag30,Volume_mean_lag3,Volume_mean_lag7,Volume_mean_lag30,Volume_std_lag3,Volume_std_lag7,Volume_std_lag30,Turnover_mean_lag3,Turnover_mean_lag7,Turnover_mean_lag30,Turnover_std_lag3,Turnover_std_lag7,Turnover_std_lag30,Trades_mean_lag3,Trades_mean_lag7,Trades_mean_lag30,Trades_std_lag3,Trades_std_lag7,Trades_std_lag30
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2011-06-01,2011-06-01,ASIANPAINT,EQ,3133.40,3145.10,3194.00,3061.50,3170.00,3169.35,3156.21,192349,6.070944e+13,11665.0,89136.0,0.4634,1865.291504,1865.337646,1865.722778,22.184906,36.629894,84.242317,1815.439453,1815.481689,1815.813965,22.479603,36.825569,84.067940,4163639.25,4162508.75,4157626.0,1.460708e+06,1.816697e+06,2.374133e+06,2.151483e+14,2.151264e+14,2.150270e+14,7.552617e+13,9.420176e+13,1.235832e+14,57895.210938,57890.550781,57866.812500,16894.664062,20467.808594,25287.337891
2011-06-02,2011-06-02,ASIANPAINT,EQ,3169.35,3159.00,3171.80,3087.00,3110.05,3105.05,3133.25,211697,6.633002e+13,6690.0,169251.0,0.7995,3194.000000,3194.000000,3194.000000,22.184906,36.629894,84.242317,3061.500000,3061.500000,3061.500000,22.479603,36.825569,84.067940,192349.00,192349.00,192349.0,1.460708e+06,1.816697e+06,2.374133e+06,6.070944e+13,6.070944e+13,6.070944e+13,7.552617e+13,9.420176e+13,1.235832e+14,11665.000000,11665.000000,11665.000000,16894.664062,20467.808594,25287.337891
2011-06-03,2011-06-03,ASIANPAINT,EQ,3105.05,3105.05,3190.00,3099.95,3181.00,3167.80,3153.77,83355,2.628821e+13,5398.0,45782.0,0.5492,3182.899902,3182.899902,3182.899902,15.697770,15.697770,15.697770,3074.250000,3074.250000,3074.250000,18.031223,18.031223,18.031223,202023.00,202023.00,202023.0,1.368110e+04,1.368110e+04,1.368110e+04,6.351973e+13,6.351973e+13,6.351973e+13,3.974345e+12,3.974345e+12,3.974345e+12,9177.500000,9177.500000,9177.500000,3517.856201,3517.856201,3517.856201
2011-06-06,2011-06-06,ASIANPAINT,EQ,3167.80,3160.20,3188.85,3141.25,3169.50,3170.05,3163.77,46291,1.464539e+13,4838.0,20367.0,0.4400,3185.266602,3185.266602,3185.266602,11.832723,11.832723,11.832723,3082.816650,3082.816650,3082.816650,19.563379,19.563379,19.563379,162467.00,162467.00,162467.0,6.919261e+04,6.919261e+04,6.919261e+04,5.110923e+13,5.110923e+13,5.110923e+13,2.167855e+13,2.167855e+13,2.167855e+13,7917.666504,7917.666504,7917.666504,3308.957031,3308.957031,3308.957031
2011-06-07,2011-06-07,ASIANPAINT,EQ,3170.05,3170.10,3229.80,3155.00,3196.00,3206.15,3202.18,82014,2.626232e+13,8868.0,27403.0,0.3341,3183.550049,3186.162598,3186.162598,10.192031,9.826102,9.826102,3109.399902,3097.425049,3097.425049,28.332710,33.298111,33.298111,113781.00,133423.00,133423.0,8.679915e+04,8.103062e+04,8.103062e+04,3.575454e+13,4.199326e+13,4.199326e+13,2.711151e+13,2.541081e+13,2.541081e+13,5642.000000,7147.750000,7147.750000,949.804199,3109.750977,3109.750977


In [9]:
df.Date = pd.to_datetime(df.Date, format="%Y-%m-%d")
df["month"] = df.Date.dt.month
df["week"] = df.Date.dt.week
df["day"] = df.Date.dt.day
df["day_of_week"] = df.Date.dt.dayofweek
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble,High_mean_lag3,High_mean_lag7,High_mean_lag30,High_std_lag3,High_std_lag7,High_std_lag30,Low_mean_lag3,Low_mean_lag7,Low_mean_lag30,Low_std_lag3,Low_std_lag7,Low_std_lag30,Volume_mean_lag3,Volume_mean_lag7,Volume_mean_lag30,Volume_std_lag3,Volume_std_lag7,Volume_std_lag30,Turnover_mean_lag3,Turnover_mean_lag7,Turnover_mean_lag30,Turnover_std_lag3,Turnover_std_lag7,Turnover_std_lag30,Trades_mean_lag3,Trades_mean_lag7,Trades_mean_lag30,Trades_std_lag3,Trades_std_lag7,Trades_std_lag30,month,week,day,day_of_week
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2011-06-01,2011-06-01,ASIANPAINT,EQ,3133.40,3145.10,3194.00,3061.50,3170.00,3169.35,3156.21,192349,6.070944e+13,11665.0,89136.0,0.4634,1865.291504,1865.337646,1865.722778,22.184906,36.629894,84.242317,1815.439453,1815.481689,1815.813965,22.479603,36.825569,84.067940,4163639.25,4162508.75,4157626.0,1.460708e+06,1.816697e+06,2.374133e+06,2.151483e+14,2.151264e+14,2.150270e+14,7.552617e+13,9.420176e+13,1.235832e+14,57895.210938,57890.550781,57866.812500,16894.664062,20467.808594,25287.337891,6,22,1,2
2011-06-02,2011-06-02,ASIANPAINT,EQ,3169.35,3159.00,3171.80,3087.00,3110.05,3105.05,3133.25,211697,6.633002e+13,6690.0,169251.0,0.7995,3194.000000,3194.000000,3194.000000,22.184906,36.629894,84.242317,3061.500000,3061.500000,3061.500000,22.479603,36.825569,84.067940,192349.00,192349.00,192349.0,1.460708e+06,1.816697e+06,2.374133e+06,6.070944e+13,6.070944e+13,6.070944e+13,7.552617e+13,9.420176e+13,1.235832e+14,11665.000000,11665.000000,11665.000000,16894.664062,20467.808594,25287.337891,6,22,2,3
2011-06-03,2011-06-03,ASIANPAINT,EQ,3105.05,3105.05,3190.00,3099.95,3181.00,3167.80,3153.77,83355,2.628821e+13,5398.0,45782.0,0.5492,3182.899902,3182.899902,3182.899902,15.697770,15.697770,15.697770,3074.250000,3074.250000,3074.250000,18.031223,18.031223,18.031223,202023.00,202023.00,202023.0,1.368110e+04,1.368110e+04,1.368110e+04,6.351973e+13,6.351973e+13,6.351973e+13,3.974345e+12,3.974345e+12,3.974345e+12,9177.500000,9177.500000,9177.500000,3517.856201,3517.856201,3517.856201,6,22,3,4
2011-06-06,2011-06-06,ASIANPAINT,EQ,3167.80,3160.20,3188.85,3141.25,3169.50,3170.05,3163.77,46291,1.464539e+13,4838.0,20367.0,0.4400,3185.266602,3185.266602,3185.266602,11.832723,11.832723,11.832723,3082.816650,3082.816650,3082.816650,19.563379,19.563379,19.563379,162467.00,162467.00,162467.0,6.919261e+04,6.919261e+04,6.919261e+04,5.110923e+13,5.110923e+13,5.110923e+13,2.167855e+13,2.167855e+13,2.167855e+13,7917.666504,7917.666504,7917.666504,3308.957031,3308.957031,3308.957031,6,23,6,0
2011-06-07,2011-06-07,ASIANPAINT,EQ,3170.05,3170.10,3229.80,3155.00,3196.00,3206.15,3202.18,82014,2.626232e+13,8868.0,27403.0,0.3341,3183.550049,3186.162598,3186.162598,10.192031,9.826102,9.826102,3109.399902,3097.425049,3097.425049,28.332710,33.298111,33.298111,113781.00,133423.00,133423.0,8.679915e+04,8.103062e+04,8.103062e+04,3.575454e+13,4.199326e+13,4.199326e+13,2.711151e+13,2.541081e+13,2.541081e+13,5642.000000,7147.750000,7147.750000,949.804199,3109.750977,3109.750977,6,23,7,1


In [10]:
from sklearn.decomposition import PCA

In [11]:
pca = PCA(random_state=42, n_components=2)

In [12]:
df_pca = pca.fit_transform(df.drop(labels=["Date", "VWAP", "Symbol", "Series", "month",	"week",	"day",	"day_of_week"], axis = 1))
df_pca.shape

(97800, 2)

In [13]:
dfpca = pd.DataFrame(df_pca)
dfpca.index = df.index
dfpca = dfpca.join(df[["Date","VWAP", "Symbol", "Series", "month",	"week",	"day",	"day_of_week"]])

In [14]:
dfpca.head()

,0,1,Date,VWAP,Symbol,Series,month,week,day,day_of_week
Date,,,,,,,,,,
2011-06-01,-7.446261e+13,-1.173779e+14,2011-06-01,3156.21,ASIANPAINT,EQ,6,22,1,2
2011-06-01,-7.446261e+13,-1.173779e+14,2011-06-01,1278.97,AXISBANK,EQ,6,22,1,2
2011-06-01,-7.446261e+13,-1.173779e+14,2011-06-01,1343.36,BAJAJ-AUTO,EQ,6,22,1,2
2011-06-01,-7.446261e+13,-1.173779e+14,2011-06-01,522.81,BAJAJFINSV,EQ,6,22,1,2
2011-06-01,-7.446261e+13,-1.173779e+14,2011-06-01,381.07,BHARTIARTL,EQ,6,22,1,2


In [15]:
dfpca.rename(columns={0: "Feature1", 1: "Feature2"}, inplace = True)
dfpca.head()

,Feature1,Feature2,Date,VWAP,Symbol,Series,month,week,day,day_of_week
Date,,,,,,,,,,
2011-06-01,-7.446261e+13,-1.173779e+14,2011-06-01,3156.21,ASIANPAINT,EQ,6,22,1,2
2011-06-01,-7.446261e+13,-1.173779e+14,2011-06-01,1278.97,AXISBANK,EQ,6,22,1,2
2011-06-01,-7.446261e+13,-1.173779e+14,2011-06-01,1343.36,BAJAJ-AUTO,EQ,6,22,1,2
2011-06-01,-7.446261e+13,-1.173779e+14,2011-06-01,522.81,BAJAJFINSV,EQ,6,22,1,2
2011-06-01,-7.446261e+13,-1.173779e+14,2011-06-01,381.07,BHARTIARTL,EQ,6,22,1,2


In [16]:
df_train = dfpca[dfpca.Date < "2019"]
df_valid = dfpca[dfpca.Date >= "2019"]

exogenous_features = ["Feature1", "Feature2"]
print(exogenous_features)

['Feature1', 'Feature2']


In [17]:
params = {"objective": "regression"}

dtrain = lgb.Dataset(df_train[exogenous_features], label=df_train.VWAP.values)
dvalid = lgb.Dataset(df_valid[exogenous_features])

model_lgb = lgb.train(params, train_set=dtrain)

forecast = model_lgb.predict(df_valid[exogenous_features])
df_valid["Forecast_LightGBM"] = forecast

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [18]:
# model = auto_arima(df_train.VWAP, exogenous=df_train[exogenous_features], 
#                    trace=True, error_action="ignore", suppress_warnings=True)
# model.fit(df_train.VWAP, exogenous=df_train[exogenous_features])

# forecast = model.predict(n_periods=len(df_valid), exogenous=df_valid[exogenous_features])
# df_valid["Forecast_ARIMAX"] = forecast

In [ ]:
# model_fbp = Prophet()
# for feature in exogenous_features:
#     model_fbp.add_regressor(feature)

# model_fbp.fit(df_train[["Date", "VWAP"] + exogenous_features].rename(columns={"Date": "ds", "VWAP": "y"}))

# forecast = model_fbp.predict(df_valid[["Date", "VWAP"] + exogenous_features].rename(columns={"Date": "ds"}))
# df_valid["Forecast_Prophet"] = forecast.yhat.values

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
